In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
import IPython
import os
from  Segmenter.utils import classname_to_color_mapper, draw_prediction_on_face
from tqdm.notebook import tqdm

def delete_all_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                delete_all_files_in_folder(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))


def imshow(img):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret,width =500,height =500)
    IPython.display.display(i)


def predict_image(model,image_name):

    # Load your RGB image

    image_1 = Image.open(f'SISTEMAS_UFF/Images/{image_name}')


    if image_name.endswith("posx"):
        pass
    elif image_name.endswith("negx"):
        pass
    elif image_name.endswith("negy"):
        image_1 = image_1.rotate(270)
        pass
    elif image_name.endswith("posy"):
        image_1 = image_1.rotate(90)
        pass
    elif image_name.endswith("posz"):
        pass
    elif image_name.endswith("negz"):
        pass

    image_1 = np.array(image_1)[..., 2::-1]



    result_of_the_segmentation = model(image_1,save=False, verbose=False)

    return draw_prediction_on_face(image_1,result_of_the_segmentation[0])
    


def predict_and_save():
    file_count = 0
    model = YOLO('runs/segment/train_export_v6_medium/weights/best.pt')  # load a pretrained model (recommended for training)
    delete_all_files_in_folder("to_metrics_v6/")
    for filename in tqdm(os.listdir("datasets/darwin_export_v6/images/test/")):
        image_name = filename

        if not image_name.endswith(".png"):
            image_name+=".png"

        mask = predict_image(model,image_name)

        im = Image.fromarray(mask)
        im.save(f"to_metrics_v6/{image_name}")
        file_count += 1
    print(file_count)
    
predict_and_save()

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
import IPython
import os
from utils import classname_to_color_mapper, draw_prediction_on_face_mask, draw_prediction_on_face
from tqdm.notebook import tqdm

def delete_all_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                delete_all_files_in_folder(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))


def imshow(img):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret,width =1000,height =1000)
    IPython.display.display(i)


def predict_image(model,image_name):

    # Load your RGB image

    image_1 = Image.open(f'SISTEMAS_UFF/Images/{image_name}')


    if image_name.endswith("posx"):
        pass
    elif image_name.endswith("negx"):
        pass
    elif image_name.endswith("negy"):
        image_1 = image_1.rotate(270)
        pass
    elif image_name.endswith("posy"):
        image_1 = image_1.rotate(90)
        pass
    elif image_name.endswith("posz"):
        pass
    elif image_name.endswith("negz"):
        pass

    image_1 = np.array(image_1)[..., 2::-1]

    result_of_the_segmentation = model(image_1,save=False, verbose=False)

    return draw_prediction_on_face(image_1,result_of_the_segmentation[0])
    
def create_face_name_list(image_name):
    img_name_posx = image_name+'_posx'
    img_name_negx = image_name+'_negx'
    img_name_posy = image_name+'_posy'
    img_name_negy = image_name+'_negy'
    img_name_posz = image_name+'_posz'
    img_name_negz = image_name+'_negz'
    return [img_name_posx,img_name_negx,img_name_posy,img_name_negy,img_name_posz,img_name_negz]

def create_full_image_from_faces(face_path_list):
    width,height = 1344,1344

    complete_image = np.zeros((height*3,width*4,3))
    for face_name in face_path_list:
        image_original = Image.open(f"SISTEMAS_UFF/Images/{face_name}.png")
        
        if face_name.lower().endswith("posx"):
            pass
        elif face_name.lower().endswith("negx"):
            pass
        elif face_name.lower().endswith("negy"):
            image_original = image_original.rotate(270)
            pass
        elif face_name.lower().endswith("posy"):
            image_original = image_original.rotate(90)
            pass
        elif face_name.lower().endswith("posz"):
            pass
        elif face_name.lower().endswith("negz"):
            pass
        
        img = np.array(image_original) 
        height_pos = 0
        width_pos = 0
        if face_name.endswith("posx"):
            height_pos = 1
            width_pos = 2
            pass
        elif face_name.endswith("negx"):
            height_pos = 1
            width_pos = 0
            pass
        elif face_name.endswith("negy"):
            height_pos = 0
            width_pos = 1
            pass
        elif face_name.endswith("posy"):
            height_pos = 2
            width_pos = 1
            pass
        elif face_name.endswith("posz"):
            height_pos = 1
            width_pos = 1
            pass
        elif face_name.endswith("negz"):
            height_pos = 1
            width_pos = 3
            pass
        complete_image[height*height_pos:height*(height_pos+1),width*width_pos:width*(width_pos+1),:] = img
    complete_image[..., :3] = complete_image[..., 2::-1]
    return complete_image

def extract_face_from_cubemap(cubemap):
    width,height = 1344,1344

    topo     = np.s_[height*0:height*1,width*1:width*2]
    esquerda = np.s_[height*1:height*2,width*0:width*1]
    centro   = np.s_[height*1:height*2,width*1:width*2]
    direita  = np.s_[height*1:height*2,width*2:width*3]
    contrario  = np.s_[height*1:height*2,width*3:width*4]
    base     = np.s_[height*2:height*3,width*1:width*2]

    lista_faces = [esquerda,centro,direita,contrario]

    face_1 = cubemap[esquerda]
    face_2 = cubemap[centro]
    face_3 = cubemap[direita]
    face_4 = cubemap[contrario]

    faces = [face_1,face_2,face_3,face_4]

    face_topo = cubemap[topo]
    face_base = cubemap[base]
    
    canvas = np.zeros((height*3,width*3,3))
    face_list = []

    for face_number in range(len(faces)):
        canvas[topo]     = np.rot90(face_topo, k=face_number, axes=(1,0))
        canvas[esquerda] = np.rot90(faces[face_number], k=0, axes=(1,0))
        canvas[centro]   = np.rot90(faces[(face_number+1)%4], k=0, axes=(1,0))
        canvas[direita]  = np.rot90(faces[(face_number+2)%4], k=0, axes=(1,0))
        canvas[base]     = np.rot90(face_base, k=-face_number, axes=(1,0))
        face_list.append(np.copy(canvas))
    
    canvas[topo]     = np.rot90(faces[3], k=2, axes=(1,0))
    canvas[esquerda] = np.rot90(faces[0], k=1, axes=(1,0))
    canvas[centro]   = np.rot90(face_topo, k=0, axes=(1,0))
    canvas[direita]  = np.rot90(faces[2], k=-1, axes=(1,0))
    canvas[base]     = np.rot90(faces[1], k=-0, axes=(1,0))
    face_list.append(np.copy(canvas))

    canvas[topo]     = np.rot90(faces[1], k=0, axes=(1,0))
    canvas[esquerda] = np.rot90(faces[0], k=-1, axes=(1,0))
    canvas[centro]   = np.rot90(face_base, k=0, axes=(1,0))
    canvas[direita]  = np.rot90(faces[2], k=1, axes=(1,0))
    canvas[base]     = np.rot90(faces[3], k=2, axes=(1,0))
    face_list.append(np.copy(canvas))

    return face_list


def predict_and_save():
    file_count = 0
    model = YOLO('runs/segment/train_export_v6_large_augmented/weights/best.pt')  # load a pretrained model (recommended for training)
    delete_all_files_in_folder("to_metrics_v6/")
    delete_all_files_in_folder("to_metrics_v6_full_image/")
    header_read = False
    width,height = 1344,1344

    vertical_padding = 200
    horizontal_padding = 200

    with open("SISTEMAS_UFF/Experimental_Sets3.csv") as split_file:
        img_list = []
        for line in tqdm(split_file):
            if header_read:
                face_id, face_name, _, face_set = line.strip().split(",")

                if int(face_set) == 3:
                    img_name = face_name[7:-9]
                    img_list.append(img_name)

            if line.strip() == ",Images,Labels,Set":
                header_read = True
        img_list = set(img_list)
    print(img_list)
    face_name_list = ["posz","posx","negz","negx","negy","posy"]


    for image_name in tqdm(img_list):
        

        face_path_list = create_face_name_list(image_name)
        complete_image = create_full_image_from_faces(face_path_list)
        face_list = extract_face_from_cubemap(complete_image)
        
        for i in range(len(face_list)):
            face = face_list[i]
            face_name = face_name_list[i]
            face_cutted = face[height*1-vertical_padding:height*2+vertical_padding,width*1-horizontal_padding:width*2+horizontal_padding]
            
            result_of_the_segmentation = model(face_cutted,save=False, verbose=False)

            mask = draw_prediction_on_face_mask(face_cutted,result_of_the_segmentation[0])
            
            im = Image.fromarray(mask[vertical_padding:height+vertical_padding,horizontal_padding:width+horizontal_padding])
            im.save(f"to_metrics_v6/{image_name}_{face_name}.png")

            """
            print(f"max := {mask[vertical_padding:height+vertical_padding,horizontal_padding:width+horizontal_padding].max()}")
            print(f"min := {mask[vertical_padding:height+vertical_padding,horizontal_padding:width+horizontal_padding].min()}")
            print(f"uniques := {np.unique(mask[vertical_padding:height+vertical_padding,horizontal_padding:width+horizontal_padding])}")
            print()
            #"""

            mask = draw_prediction_on_face(face_cutted,result_of_the_segmentation[0])
            
            im = Image.fromarray(mask[vertical_padding:height+vertical_padding,horizontal_padding:width+horizontal_padding])
            im.save(f"to_metrics_v6_full_image/{image_name}_{face_name}.png")
            file_count += 1

            
        
    print(file_count)
    
predict_and_save()

0it [00:00, ?it/s]

{'cubemap_P74_e78aecf9b104474f1aa16a76b98702b1', 'cubemap_P74_e08f08fc0d77e6f74ba1d60253c632f9', 'cubemap_P74_0397e81e415ba4b9018ed9680648be41', 'cubemap_P74_a5859d67c047bc97df9f718622e65a0e', 'cubemap_P74_d9733507c062de2ec6e7a601bc253c01', 'cubemap_P74_a36a1a90b0a277fdfa89b3e7d934e70f', 'cubemap_P74_08138abfa23b421ccde6d69d26e07c01', 'cubemap_P74_5d1a4ffbcb8b43d55bb8af35519d6c41', 'cubemap_P74_e13ecefea0bdab8d2c577818bda7dca6', 'cubemap_P74_99d653c84e0c0e9eba36c59f2ac3dde2', 'cubemap_P74_23804c4c3a3746cee6821bd517eeddeb', 'cubemap_P74_d498d65f3f9293cf3b5baff3860c5045', 'cubemap_P74_a9cc4f856fac2efaa9caafa146bca401', 'cubemap_P74_c2c111d141f7f07f6455c3fe50b704ef', 'cubemap_P74_cf4a4d710e66166000ac0662de1bc9c7', 'cubemap_P74_d26fc5fd2df4e071d51917f3eb8f0038', 'cubemap_P74_ed7435d1327844d82985959cfc6ab7b8', 'cubemap_P74_37531c118855f42843521926854ad7cf', 'cubemap_P74_450f90937f67d7f9b718ae366e5e7da1', 'cubemap_P74_bbbe5b7e4493442f2b069645c4882a93', 'cubemap_P74_6d9b9023c289c38dd88002e146

  0%|          | 0/48 [00:00<?, ?it/s]

288
